# Generating and visualizing control experiments results in the paper: "Cochlear aqueduct advection and diffusion inferred from computed tomography imaging with a Bayesian approach"

## Run the code to generate the control experiments results
You can run the code for the control Bayesian inference experiment (presented in figure 2 in the paper) using the following command lines (documentation of the command line arguments can be found in the `README.md` file):


In [ ]:
max_depth = 1 # Depth of the NUTS tree, was set to 10 in the paper.
Ns = 10 # Number of samples to draw from the posterior distribution, was set to 2000 in the paper.
Nb = 2 # Number of burn-in samples, was set to 20 in the paper.

command_fig2_row1 = f"""
python code/advection_diffusion_inference.py -animal m1 -ear r -version control1 -sampler NUTSWithGibbs -unknown_par_type synth_diff1.npz -unknown_par_value 100.0 -data_type synthetic -inference_type heterogeneous -Ns {Ns} -Nb {Nb} -noise_level std_0.5 -add_data_pts -num_CA 5 -num_ST 0 -rbc fromDataClip -NUTS_kwargs '{{"max_depth": {max_depth}, "step_size": 0.1}}' --data_grad  --sampler_callback  --adaptive"""
!{command_fig2_row1}

true_a = -1.0
command = lambda max_depth, Ns, Nb, true_a: f"""
python code/advection_diffusion_inference.py -animal m1 -ear r -version control1 -sampler NUTSWithGibbs -unknown_par_type synth_diff1.npz -unknown_par_value 100.0 -data_type synthetic -inference_type advection_diffusion -Ns {Ns} -Nb {Nb} -noise_level std_0.5 -add_data_pts -num_CA 5 -num_ST 0 -true_a {true_a} -rbc fromDataClip -NUTS_kwargs '{{"max_depth": {max_depth}, "step_size": 0.1}}' --data_grad  --sampler_callback  --adaptive"""
command_fig2_row2 = command(max_depth, Ns, Nb, true_a)
#!{command_fig2_row2}

true_a = 0.5
command_fig2_row3 = command(max_depth, Ns, Nb, true_a)
#!{command_fig2_row3}

true_a = 2.0
command_fig2_row4 = command(max_depth, Ns, Nb, true_a)
#!{command_fig2_row4}

Arguments:
Namespace(animal='m1', ear='r', version='control1', sampler='NUTSWithGibbs', unknown_par_type='synth_diff1.npz', unknown_par_value=['100.0'], data_type='syntheticFromDiffusion', inference_type='heterogeneous', Ns=10, Nb=2, noise_level='std_0.5', add_data_pts=[], num_CA=5, num_ST=0, true_a=None, rbc='fromDataClip', adaptive=True, NUTS_kwargs={'max_depth': 1, 'step_size': 0.1}, data_grad=True, u0_from_data=False, sampler_callback=True, pixel_data=False)
Tag:
m1_r_NUTSWithGibbs_synth_diff1.npz_100.0_syntheticFromDiffusion_heterogeneous_10_std_0.5_control1__0_5_none_fromDataClip
Reading data from:  synth_diff1.npz
Sample: 100%|███████████████████████████████████| 10/10 [00:26<00:00,  2.62s/it]
Reading file:  results/control1/outputm1_r_NUTSWithGibbs_synth_diff1.npz_100.0_syntheticFromDiffusion_heterogeneous_10_std_0.5_control1__0_5_none_fromDataClip/m1_r_NUTSWithGibbs_synth_diff1.npz_100.0_syntheticFromDiffusion_heterogeneous_10_std_0.5_control1__0_5_none_fromDataClip_var.pkl


**Note:** to generate the appedix figures A1 and A2, you only need to replace `synth_diff1.npz` with `synth_diff2.npz` and `synth_diff3.npz`, respectively, in the command line arguments to get the corresponding "true" diffusion coefficients. You can also update the `-version` argument to `control2` and `control3`, respectively, for better organization of the results.

## Visualize the results

In [ ]:
scenarios_dir =  ["/Users/amal/Documents/research_code/CUQI-DTU/Collab-BrainEfflux-Data/A_Considered_for_the_paper/results_feb6"]

In [ ]:
option = 0
# Read case diff 1
scenarios_subdir_1 = [
                    ["paperV4CASynthDiff_Gauess_gibbs_scale_all_diff1_zerou0_update_hp_5std",
                     "paperV4CASynthAdvDiff_Gauess_gibbs_scale_all_diff1_zerou0_update_hp_5std",
                     "paperV4CASynthAdvDiff_Gauess_gibbs_scale_all_diff1_zerou0_update_hp_5std",
                     "paperV4CASynthAdvDiff_Gauess_gibbs_scale_all_diff1_zerou0_update_hp_5std"]]

scenario_tags_list_1 = [
    ["m1_r_NUTSWithGibbs_synth_diff1.npz_100.0_syntheticFromDiffusion_heterogeneous_2000_std_0.5_paperV4CASynthDiff_Gauess_gibbs_scale_all_diff1_zerou0_update_hp_5std__0_5_0.1_fromDataClip",
     "m1_r_NUTSWithGibbs_synth_diff1.npz_100.0_syntheticFromDiffusion_advection_diffusion_2000_std_0.5_paperV4CASynthAdvDiff_Gauess_gibbs_scale_all_diff1_zerou0_update_hp_5std__0_5_-1.0_fromDataClip",
     "m1_r_NUTSWithGibbs_synth_diff1.npz_100.0_syntheticFromDiffusion_advection_diffusion_2000_std_0.5_paperV4CASynthAdvDiff_Gauess_gibbs_scale_all_diff1_zerou0_update_hp_5std__0_5_0.5_fromDataClip",
     "m1_r_NUTSWithGibbs_synth_diff1.npz_100.0_syntheticFromDiffusion_advection_diffusion_2000_std_0.5_paperV4CASynthAdvDiff_Gauess_gibbs_scale_all_diff1_zerou0_update_hp_5std__0_5_2.0_fromDataClip"]]

In [ ]:
# read case diff 2
scenarios_subdir_2 = [
                    ["paperV4CASynthDiff_Gauess_gibbs_scale_all_diff2_zerou0_update_hp_5std",
                     "paperV4CASynthAdvDiff_Gauess_gibbs_scale_all_diff2_zerou0_update_hp_5std",
                     "paperV4CASynthAdvDiff_Gauess_gibbs_scale_all_diff2_zerou0_update_hp_5std",
                     "paperV4CASynthAdvDiff_Gauess_gibbs_scale_all_diff2_zerou0_update_hp_5std"]]

scenario_tags_list_2 = [
    ["m3_l_NUTSWithGibbs_synth_diff2.npz_100.0_syntheticFromDiffusion_heterogeneous_2000_std_0.5_paperV4CASynthDiff_Gauess_gibbs_scale_all_diff2_zerou0_update_hp_5std__0_5_0.1_fromDataClip",
     "m3_l_NUTSWithGibbs_synth_diff2.npz_100.0_syntheticFromDiffusion_advection_diffusion_2000_std_0.5_paperV4CASynthAdvDiff_Gauess_gibbs_scale_all_diff2_zerou0_update_hp_5std__0_5_-1.0_fromDataClip",
     "m3_l_NUTSWithGibbs_synth_diff2.npz_100.0_syntheticFromDiffusion_advection_diffusion_2000_std_0.5_paperV4CASynthAdvDiff_Gauess_gibbs_scale_all_diff2_zerou0_update_hp_5std__0_5_0.5_fromDataClip",
     "m3_l_NUTSWithGibbs_synth_diff2.npz_100.0_syntheticFromDiffusion_advection_diffusion_2000_std_0.5_paperV4CASynthAdvDiff_Gauess_gibbs_scale_all_diff2_zerou0_update_hp_5std__0_5_2.0_fromDataClip"]]



In [ ]:
# read case diff 3
scenarios_subdir_3 = [
                    ["paperV4CASynthDiff_Gauess_gibbs_scale_all_diff3_zerou0_update_hp_5std",
                     "paperV4CASynthAdvDiff_Gauess_gibbs_scale_all_diff3_zerou0_update_hp_5std",
                     "paperV4CASynthAdvDiff_Gauess_gibbs_scale_all_diff3_zerou0_update_hp_5std",
                     "paperV4CASynthAdvDiff_Gauess_gibbs_scale_all_diff3_zerou0_update_hp_5std"]]

scenario_tags_list_3 = [
    ["m1_r_NUTSWithGibbs_synth_diff3.npz_100.0_syntheticFromDiffusion_heterogeneous_2000_std_0.5_paperV4CASynthDiff_Gauess_gibbs_scale_all_diff3_zerou0_update_hp_5std__0_5_0.1_fromDataClip",
        "m1_r_NUTSWithGibbs_synth_diff3.npz_100.0_syntheticFromDiffusion_advection_diffusion_2000_std_0.5_paperV4CASynthAdvDiff_Gauess_gibbs_scale_all_diff3_zerou0_update_hp_5std__0_5_-1.0_fromDataClip",
        "m1_r_NUTSWithGibbs_synth_diff3.npz_100.0_syntheticFromDiffusion_advection_diffusion_2000_std_0.5_paperV4CASynthAdvDiff_Gauess_gibbs_scale_all_diff3_zerou0_update_hp_5std__0_5_0.5_fromDataClip",
        "m1_r_NUTSWithGibbs_synth_diff3.npz_100.0_syntheticFromDiffusion_advection_diffusion_2000_std_0.5_paperV4CASynthAdvDiff_Gauess_gibbs_scale_all_diff3_zerou0_update_hp_5std__0_5_2.0_fromDataClip"]]

# e72362512cb8bf8fa68931bdadfdf6d33c4bad6f paperV4CASynthDiff_Gauess_gibbs_scale_all_diff3


In [ ]:
# create plots directory 
import os
if not os.path.exists('figures/control'):
    os.makedirs('figures/control')
color_map_string = 'coolwarm' #'twilight' #'terrain'#'coolwarm'#'berlin' #'brg'
colormap=plt.colormaps.get_cmap(color_map_string)

# plot control case diff 1
data_list_diff1 = read_all_scenarios(scenarios_dir[option], scenarios_subdir_1[option], scenario_tags_list_1[option])
plot_type='over_time' # over_time or over_location
plot_control_case_v2(data_list_diff1, plot_type=plot_type, colormap=colormap)
plt.savefig('figures/control/control_diff1_v2_'+str(option)+'.pdf')

# plot control case diff 2
data_list_diff2 = read_all_scenarios(scenarios_dir[option], scenarios_subdir_2[option], scenario_tags_list_2[option])
plot_control_case_v2(data_list_diff2, plot_type=plot_type, colormap=colormap, d_y_coor=0.5)
plt.savefig('figures/control/control_diff2_v2_'+str(option)+'.pdf')

## plot control case diff 3
data_list_diff3 = read_all_scenarios(scenarios_dir[option], scenarios_subdir_3[option], scenario_tags_list_3[option])
plot_control_case_v2(data_list_diff3, plot_type=plot_type, colormap=colormap, d_y_coor=0.3)
plt.savefig('figures/control/control_diff3_v2_'+str(option)+'.pdf')


In [ ]:
Ess_global = []
for i in range(4):
    Ess_global.extend(data_list_diff1[i]['x_samples'].compute_ess().tolist())
    Ess_global.extend(data_list_diff2[i]['x_samples'].compute_ess().tolist())
    Ess_global.extend(data_list_diff3[i]['x_samples'].compute_ess().tolist())

Ess_global = np.array(Ess_global)
print(Ess_global.min())
print(Ess_global.max())
print(Ess_global.mean())

In [ ]:
data_list_diff1[i]['experiment_par']
